* Iterate over folders and resize all volumetric images to 96x96 and save in two folders
* Load all data as numpy array (Samples, 96, 96, 263, 1), (Samples, 2)
* Train

In [1]:
from keras.utils import np_utils
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
from keras.models import Model
import cv2
import os
import numpy as np
import keras
import h5py

Using TensorFlow backend.


In [13]:
def one_hot(y_train):
    y_train = np_utils.to_categorical(y_train,2)
    print(y_train.shape)
    return y_train

def load_numpy(path):
    val = np.load(path)
    print(val.shape)
    return val

def expand_dims(val):
    val_exp = np.expand_dims(val, axis=4)
    return val_exp


def plot_slice(img, sl_num):
    plt.imshow(img[:,:,sl_num])
    

w, h = 96, 96
def rs_img(img):
    flatten = [cv2.resize(img[:,:,i], (w, h), interpolation=cv2.INTER_CUBIC) for i in range(img.shape[-1])]
    img = np.array(np.dstack(flatten)) 
    return img

In [3]:
print(len(os.listdir("/home/hasib/imageclef/TB/training/High/")))
print(len(os.listdir("/home/hasib/imageclef/TB/training/Low/")))

107
111


In [4]:
paths_high = "/home/hasib/imageclef/TB/training/High/"
paths_low = "/home/hasib/imageclef/TB/training/Low/"

In [5]:
c = 0
for path in os.listdir(paths_high)[:]:
    
    img_3d = None
    img_3d_rs = None
    
    img_3d = np.load(paths_high+ "{}".format(path))
    img_3d_rs = rs_img(img_3d)
    c+=1
    print(c,"-----", img_3d.shape, "-------", img_3d_rs.shape)
    np.save("/home/hasib/imageclef/TB/training_96/High/{}_rs.npy".format(c), img_3d_rs)

1 ----- (512, 512, 263) ------- (96, 96, 263)
2 ----- (512, 512, 263) ------- (96, 96, 263)
3 ----- (512, 512, 263) ------- (96, 96, 263)
4 ----- (512, 512, 263) ------- (96, 96, 263)
5 ----- (512, 512, 263) ------- (96, 96, 263)
6 ----- (512, 512, 263) ------- (96, 96, 263)
7 ----- (512, 512, 263) ------- (96, 96, 263)
8 ----- (512, 512, 263) ------- (96, 96, 263)
9 ----- (512, 512, 263) ------- (96, 96, 263)
10 ----- (512, 512, 263) ------- (96, 96, 263)
11 ----- (512, 512, 263) ------- (96, 96, 263)
12 ----- (512, 512, 263) ------- (96, 96, 263)
13 ----- (512, 512, 263) ------- (96, 96, 263)
14 ----- (512, 512, 263) ------- (96, 96, 263)
15 ----- (512, 512, 263) ------- (96, 96, 263)
16 ----- (512, 512, 263) ------- (96, 96, 263)
17 ----- (512, 512, 263) ------- (96, 96, 263)
18 ----- (512, 512, 263) ------- (96, 96, 263)
19 ----- (512, 512, 263) ------- (96, 96, 263)
20 ----- (512, 512, 263) ------- (96, 96, 263)
21 ----- (512, 512, 263) ------- (96, 96, 263)
22 ----- (512, 512, 26

In [9]:
c = 0
for path in os.listdir(paths_low)[:]:
    
    img_3d = None
    img_3d_rs = None
    img_3d = np.load(paths_low+ "{}".format(path))
    img_3d_rs = rs_img(img_3d)
    c+=1
    print(c,"-----", img_3d.shape, "-------", img_3d_rs.shape)
    np.save("/home/hasib/imageclef/TB/training_96/Low/{}_rs.npy".format(c), img_3d_rs)

1 ----- (512, 512, 263) ------- (96, 96, 263)
2 ----- (512, 512, 263) ------- (96, 96, 263)
3 ----- (512, 512, 263) ------- (96, 96, 263)
4 ----- (512, 512, 263) ------- (96, 96, 263)
5 ----- (512, 512, 263) ------- (96, 96, 263)
6 ----- (512, 512, 263) ------- (96, 96, 263)
7 ----- (512, 512, 263) ------- (96, 96, 263)
8 ----- (512, 512, 263) ------- (96, 96, 263)
9 ----- (512, 512, 263) ------- (96, 96, 263)
10 ----- (512, 512, 263) ------- (96, 96, 263)
11 ----- (512, 512, 263) ------- (96, 96, 263)
12 ----- (512, 512, 263) ------- (96, 96, 263)
13 ----- (512, 512, 263) ------- (96, 96, 263)
14 ----- (512, 512, 263) ------- (96, 96, 263)
15 ----- (512, 512, 263) ------- (96, 96, 263)
16 ----- (512, 512, 263) ------- (96, 96, 263)
17 ----- (512, 512, 263) ------- (96, 96, 263)
18 ----- (512, 512, 263) ------- (96, 96, 263)
19 ----- (512, 512, 263) ------- (96, 96, 263)
20 ----- (512, 512, 263) ------- (96, 96, 263)
21 ----- (512, 512, 263) ------- (96, 96, 263)
22 ----- (512, 512, 26

In [10]:
high_label = np.array([1 for _ in range(len(os.listdir(paths_high)))])
low_label = np.array([0 for _ in range(len(os.listdir(paths_low)))])

high_label.shape, low_label.shape

((107,), (111,))

In [11]:
y_all = np.concatenate((high_label, low_label), axis=0)
y_all.shape

(218,)

In [14]:
y_train = one_hot(y_all)
y_train.shape

(218, 2)


(218, 2)

In [15]:
np.save("y_train.npy", y_train)

In [39]:
paths_high_rs = "/home/hasib/imageclef/TB/training_96/High/"
paths_low_rs = "/home/hasib/imageclef/TB/training_96/Low/"

In [30]:
os.listdir(paths_high_rs)

['21_rs.npy',
 '32_rs.npy',
 '50_rs.npy',
 '107_rs.npy',
 '71_rs.npy',
 '91_rs.npy',
 '5_rs.npy',
 '67_rs.npy',
 '54_rs.npy',
 '38_rs.npy',
 '93_rs.npy',
 '47_rs.npy',
 '1_rs.npy',
 '83_rs.npy',
 '8_rs.npy',
 '96_rs.npy',
 '35_rs.npy',
 '40_rs.npy',
 '105_rs.npy',
 '94_rs.npy',
 '70_rs.npy',
 '25_rs.npy',
 '62_rs.npy',
 '97_rs.npy',
 '12_rs.npy',
 '79_rs.npy',
 '4_rs.npy',
 '87_rs.npy',
 '45_rs.npy',
 '64_rs.npy',
 '86_rs.npy',
 '49_rs.npy',
 '106_rs.npy',
 '52_rs.npy',
 '19_rs.npy',
 '85_rs.npy',
 '3_rs.npy',
 '61_rs.npy',
 '9_rs.npy',
 '63_rs.npy',
 '23_rs.npy',
 '16_rs.npy',
 '24_rs.npy',
 '46_rs.npy',
 '41_rs.npy',
 '51_rs.npy',
 '89_rs.npy',
 '66_rs.npy',
 '11_rs.npy',
 '55_rs.npy',
 '75_rs.npy',
 '34_rs.npy',
 '100_rs.npy',
 '76_rs.npy',
 '13_rs.npy',
 '30_rs.npy',
 '74_rs.npy',
 '28_rs.npy',
 '15_rs.npy',
 '102_rs.npy',
 '36_rs.npy',
 '22_rs.npy',
 '39_rs.npy',
 '69_rs.npy',
 '88_rs.npy',
 '6_rs.npy',
 '82_rs.npy',
 '48_rs.npy',
 '90_rs.npy',
 '18_rs.npy',
 '60_rs.npy',
 '73_rs.

In [34]:
all_highs = []
c = 0
for path in os.listdir(paths_high_rs)[:]:
    img_3d = None
    img_3d = np.load(paths_high_rs+ "{}".format(path))
    all_highs.append(img_3d)
    c+=1
    print(c, "----", paths_high_rs+path, img_3d.shape)

1 ---- /home/hasib/imageclef/TB/training_96/High/21_rs.npy (96, 96, 263)
2 ---- /home/hasib/imageclef/TB/training_96/High/32_rs.npy (96, 96, 263)
3 ---- /home/hasib/imageclef/TB/training_96/High/50_rs.npy (96, 96, 263)
4 ---- /home/hasib/imageclef/TB/training_96/High/107_rs.npy (96, 96, 263)
5 ---- /home/hasib/imageclef/TB/training_96/High/71_rs.npy (96, 96, 263)
6 ---- /home/hasib/imageclef/TB/training_96/High/91_rs.npy (96, 96, 263)
7 ---- /home/hasib/imageclef/TB/training_96/High/5_rs.npy (96, 96, 263)
8 ---- /home/hasib/imageclef/TB/training_96/High/67_rs.npy (96, 96, 263)
9 ---- /home/hasib/imageclef/TB/training_96/High/54_rs.npy (96, 96, 263)
10 ---- /home/hasib/imageclef/TB/training_96/High/38_rs.npy (96, 96, 263)
11 ---- /home/hasib/imageclef/TB/training_96/High/93_rs.npy (96, 96, 263)
12 ---- /home/hasib/imageclef/TB/training_96/High/47_rs.npy (96, 96, 263)
13 ---- /home/hasib/imageclef/TB/training_96/High/1_rs.npy (96, 96, 263)
14 ---- /home/hasib/imageclef/TB/training_96/Hig

In [35]:
len(all_highs)

106

In [36]:
all_highs = np.array(all_highs)

NameError: name 'all_high' is not defined

In [37]:
all_highs.shape

(106, 96, 96, 263)

In [40]:
all_lows = []
c = 0
for path in os.listdir(paths_low_rs)[:]:
    img_3d = None
    img_3d = np.load(paths_low_rs+ "{}".format(path))
    all_lows.append(img_3d)
    c+=1
    print(c, "----", paths_low_rs+img, img_3d.shape)

1 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
2 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
3 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
4 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
5 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
6 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
7 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
8 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
9 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
10 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
11 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
12 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
13 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (96, 96, 263)
14 ---- /home/hasib/imageclef/TB/training_96/Low/21_rs.npy (

In [41]:
len(all_lows)

111

In [ ]:
all_lows = np.array(all_lows)
all_lows.shape

In [10]:
def m_naive():
    '''playing function'''
    input_layer = Input((96, 96, 263, 1)) # 1 is just dummy dimension good for nothing 
    
    conv_layer1 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(input_layer)
    pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer1)

        
    conv_layer2 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
    pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

    conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer2)
    pooling_layer3 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer3)

    
    conv_layer4 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer3)
    pooling_layer4 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

    conv_layer5 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer4)
    pooling_layer5 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer5)
    
    pooling_layer9 = BatchNormalization()(pooling_layer5)
    flatten_layer = Flatten()(pooling_layer9)

    dense_layer1 = Dense(units=1028, activation='relu')(flatten_layer)
    dense_layer1 = Dropout(0.4)(dense_layer1)
    
    
    dense_layer2 = Dense(units=1028, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.4)(dense_layer2)
    
    dense_layer3 = Dense(units=512, activation='relu')(dense_layer2)
    dense_layer3 = Dropout(0.4)(dense_layer3)

    dense_layer4 = Dense(units=256, activation='relu')(dense_layer2)
    dense_layer4 = Dropout(0.4)(dense_layer3)

    output_layer = Dense(units=2, activation='softmax')(dense_layer4)

    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss=categorical_crossentropy, optimizer=Adadelta(lr=0.1), metrics=['acc']) # # optimizers.SGD(lr=1e-4, momentum=0.9)
    
    return model

model = None
model = m_naive()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 263, 1)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 94, 94, 261, 32)   896       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 47, 47, 130, 32)   0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 45, 45, 128, 32)   27680     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 22, 22, 64, 32)    0         
_________________________________________________________________
conv3d_3 (Conv3D)    

In [11]:
#model.fit(x=x_train_new, y=y_train, batch_size=None, epochs=20, verbose=1)